# Score frequencies

In [1]:
import sys
sys.path.insert(0, '../../src')

import json
import random
from time import perf_counter
from collections import defaultdict

from farkle.logic import gameobjects as go

In [2]:
# for saving dicehands as json
class DiceHandEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, go.DiceHand):
            return obj.json_encode
        return super().default(obj)
    
def as_dicehand(dct):
    if '__DiceHand__' in dct:
        return go.DiceHand.json_decode(dct)
    return dct

def hash_frequencies(possible_score_frequencies):
    hash_to_dh = {}
    hash_to_freq = {}
    for d in possible_score_frequencies.keys():
        hash_to_freq[d] = {}
        for dh, freq in possible_score_frequencies[d].items():
            hash_to_dh[str(hash(dh))] = dh
            hash_to_freq[d][str(hash(dh))] = freq
    return hash_to_dh, hash_to_freq

def unhash_frequencies(hash_to_dh, hash_to_freq):
    _possible_score_freq = {}
    for d in hash_to_freq:
        d_int = int(d)
        _possible_score_freq[d_int] = defaultdict(int)
        for h, freq in hash_to_freq[d].items():
            if hash_to_dh[h] is not None:
                dh = go.DiceHand.json_decode(hash_to_dh[h])
            else:
                dh = None
            _possible_score_freq[d_int][dh] = int(freq)
    return _possible_score_freq

def json_encode(possible_score_frequencies):
    hash_to_dh, hash_to_freq = hash_frequencies(possible_score_frequencies)
    d = {'hash_to_dh': hash_to_dh, 
         'hash_to_freq': hash_to_freq}
    return json.dumps(d, cls=DiceHandEncoder)

def save_freq(path: str, possible_score_frequencies: dict):
    j = json_encode(possible_score_frequencies)
    with open(path, 'w') as f:
        f.write(j)

def load_freq(path: str):
    with open(path, 'r') as f:
        j = json.loads(f.read())

    return unhash_frequencies(j['hash_to_dh'], j['hash_to_freq'])

In [4]:
# this will only return unique combinations of dice
# so has 1,2 but not 2,1
# but think I have to count duplicates for frequencies to be correct

l1 = [','.join([str(i)]) for i in range(1, 6+1)]
# l1

l2 = [','.join([str(j)] + i.split(',')) 
      for j in range(1, 6+1) 
      for i in l1 
      if j >= max([int(i) for i in i.split(',')])]
# l2

In [5]:
num_dice = 6
possible_score_frequencies = {i: defaultdict(int) for i in range(1, num_dice+1)}

# all possible 1 die combinations
hand_size = 1
hand_size_dice_hands = [[i] for i in range(1, num_dice+1)]

for hand_size in range(1, num_dice+1):
    dv_count = 0
    for dice_vals in hand_size_dice_hands:
        dh = go.DiceHand(dice_vals)
        if not dh.farkled:
            for ps in dh.possible_scores():
                possible_score_frequencies[hand_size][ps] += 1
        else:
            possible_score_frequencies[hand_size][None] += 1
        
        dv_count += 1
        write_str = f'\r{hand_size} dice | '
        write_str += f'{"#" * int(dv_count * 20 / len(hand_size_dice_hands))}'
        write_str += f'{" " * (20 - int(dv_count * 20 / len(hand_size_dice_hands)))} | '
        write_str += f'{int(dv_count * 100 / len(hand_size_dice_hands))}%'
        sys.stdout.write(write_str)
    
    hand_size_dice_hands = [[j] + i for j in range(1, num_dice+1) for i in hand_size_dice_hands]
    
    sys.stdout.write('\n')
    
freq_path = '../../../models/possible_score_frequencies.json'
save_freq(freq_path, possible_score_frequencies)

1 dice | #################### | 100%
2 dice | #################### | 100%
3 dice | #################### | 100%
4 dice | #################### | 100%
5 dice | #################### | 100%
6 dice | #################### | 100%


In [6]:
possible_score_frequencies

{1: defaultdict(int,
             {DiceHand(free=[1], locked=[], score=100): 1,
              None: 4,
              DiceHand(free=[5], locked=[], score=50): 1}),
 2: defaultdict(int,
             {DiceHand(free=[1, 1], locked=[], score=200): 1,
              DiceHand(free=[1], locked=[], score=100): 11,
              DiceHand(free=[5], locked=[], score=50): 11,
              DiceHand(free=[1, 5], locked=[], score=150): 2,
              None: 16,
              DiceHand(free=[5, 5], locked=[], score=100): 1}),
 3: defaultdict(int,
             {DiceHand(free=[1, 1, 1], locked=[], score=1000): 1,
              DiceHand(free=[1, 1], locked=[], score=200): 16,
              DiceHand(free=[1], locked=[], score=100): 91,
              DiceHand(free=[1, 1, 1], locked=[], score=300): 1,
              DiceHand(free=[1, 1, 5], locked=[], score=250): 3,
              DiceHand(free=[5], locked=[], score=50): 91,
              DiceHand(free=[1, 5], locked=[], score=150): 30,
              DiceHand(